In [2]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_databse.csv")
city_data_df["Rain (inches)"] = city_data_df["Rain (inches)"].astype("float64")
city_data_df["Snow (inches)"] = city_data_df["Snow (inches)"].astype("float64")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Ushuaia,AR,-54.80,-68.30,37.40,69,20,3.36,few clouds,0.00,0.0
1,1,Georgetown,MY,5.41,100.34,80.60,88,40,4.70,scattered clouds,0.00,0.0
2,2,Palmas Bellas,PA,9.23,-80.09,86.00,70,69,11.99,light rain,0.52,0.0
3,3,Ahipara,NZ,-35.17,173.17,50.52,79,65,2.01,broken clouds,0.00,0.0
4,4,Constitucion,CL,-35.33,-72.42,53.76,64,83,4.94,broken clouds,0.00,0.0


In [4]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain (inches)          float64
Snow (inches)          float64
dtype: object

In [5]:
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_ques = str(input("Do you want it to be raining? (yes/no) "))
snow_ques = str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [6]:
if rain_ques == 'no':
    rain_criteria = (city_data_df["Rain (inches)"] == 0.0)
else:
    rain_criteria = (city_data_df["Rain (inches)"] > 0.0)
    
if snow_ques == 'no':
    snow_criteria = (city_data_df["Snow (inches)"] == 0.0)
else:
    snow_criteria = (city_data_df["Snow (inches)"] > 0.0)

# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       rain_criteria & snow_criteria]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
1,1,Georgetown,MY,5.41,100.34,80.60,88,40,4.70,scattered clouds,0.0,0.0
12,12,Impfondo,CG,1.64,18.07,78.98,66,100,0.58,overcast clouds,0.0,0.0
15,15,Gillette,US,44.29,-105.50,81.00,36,20,5.82,few clouds,0.0,0.0
19,19,Westport,US,41.14,-73.36,88.00,66,1,6.93,clear sky,0.0,0.0
23,23,Vaini,TO,-21.20,-175.20,73.40,88,90,8.05,overcast clouds,0.0,0.0
30,30,Cefalu,IT,38.04,14.02,73.99,87,1,5.01,clear sky,0.0,0.0
35,35,Tianpeng,CN,30.99,103.94,81.00,1,100,2.39,overcast clouds,0.0,0.0
38,38,Hasaki,JP,35.73,140.83,73.40,100,75,3.36,broken clouds,0.0,0.0
40,40,Tebingtinggi,ID,3.33,99.16,78.80,88,40,3.36,scattered clouds,0.0,0.0
43,43,Makakilo City,US,21.35,-158.09,84.20,58,75,9.17,broken clouds,0.0,0.0


In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Georgetown,MY,80.60,scattered clouds,5.41,100.34,
12,Impfondo,CG,78.98,overcast clouds,1.64,18.07,
15,Gillette,US,81.00,few clouds,44.29,-105.50,
19,Westport,US,88.00,clear sky,41.14,-73.36,
23,Vaini,TO,73.40,overcast clouds,-21.20,-175.20,
30,Cefalu,IT,73.99,clear sky,38.04,14.02,
35,Tianpeng,CN,81.00,overcast clouds,30.99,103.94,
38,Hasaki,JP,73.40,broken clouds,35.73,140.83,
40,Tebingtinggi,ID,78.80,scattered clouds,3.33,99.16,
43,Makakilo City,US,84.20,broken clouds,21.35,-158.09,


In [8]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Georgetown,MY,80.60,scattered clouds,5.41,100.34,Cititel Penang
12,Impfondo,CG,78.98,overcast clouds,1.64,18.07,Préfecture de la Likouala
15,Gillette,US,81.00,few clouds,44.29,-105.50,National 9 Inn - Gillette WY
19,Westport,US,88.00,clear sky,41.14,-73.36,The Westport Inn
23,Vaini,TO,73.40,overcast clouds,-21.20,-175.20,Keleti Beach Resort
30,Cefalu,IT,73.99,clear sky,38.04,14.02,Hotel La Giara
35,Tianpeng,CN,81.00,overcast clouds,30.99,103.94,Shuangliu Love Home Business Hotel
38,Hasaki,JP,73.40,broken clouds,35.73,140.83,Inubosaki Kanko Hotel
40,Tebingtinggi,ID,78.80,scattered clouds,3.33,99.16,Hotel Malibou
43,Makakilo City,US,84.20,broken clouds,21.35,-158.09,Marriott's Ko Olina Beach Club


In [10]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [11]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [12]:
# Add a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
travel_cities = ["Vierzon","Saint-Orens-De-Gameville","Vila","Lata"]

city1_df = hotel_df.loc[(city_data_df["City"] == travel_cities[0])]
city2_df = hotel_df.loc[(city_data_df["City"] == travel_cities[1])]
city3_df = hotel_df.loc[(city_data_df["City"] == travel_cities[2])]
city4_df = hotel_df.loc[(city_data_df["City"] == travel_cities[3])]

travel_df = pd.concat([city1_df, city2_df, city3_df, city4_df])
travel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
305,Vierzon,FR,82.40,clear sky,47.21,2.06,Arche Hôtel Vierzon
245,Saint-Orens-De-Gameville,FR,86.00,scattered clouds,43.55,1.53,Appart'City Confort Toulouse Diagora Labège - ...
519,Vila,PT,75.99,clear sky,42.03,-8.16,Hotel Castrum Villae
386,Lata,PT,73.99,clear sky,40.16,-8.33,Casa Maquia


In [14]:
# Create the output file (CSV).
output_travel_cities = "analysis/Four_cities.csv"
# Export the City_Data into a CSV.
travel_df.to_csv(output_travel_cities)